In [65]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import lsqr
from sklearn.metrics import mean_squared_error
import scipy.sparse as sps

In [66]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
trainingset_file = 'ml-100k/u3.base'
testset_file= 'ml-100k/u3.test'
n_users = 943
n_items = 1682
ratings = np.zeros((n_users, n_items))

df = pd.read_csv(trainingset_file, sep='\t', names=names)

TIME_RANGE_DIVIDEND = 5

min_time = None
max_time = None
for row in df.itertuples():
    if min_time == None:
        min_time = row[4]
        max_time = row[4]
    
    if row[4] < min_time:
        min_time = row[4]
    
    if row[4] > max_time:
        max_time = row[4]

# need to plus for edge case, since for the max_time - min_time / 5 should be less than 5
time_range_length = (int)((max_time - min_time) / TIME_RANGE_DIVIDEND) + 1


vector_b = [[] for y in range(TIME_RANGE_DIVIDEND)]

row_index = [[] for y in range(TIME_RANGE_DIVIDEND)]
col_index = [[] for y in range(TIME_RANGE_DIVIDEND)]
rate_list = [[] for y in range(TIME_RANGE_DIVIDEND)]

## preserve the data base on timestamp
data_list = [[] for y in range(TIME_RANGE_DIVIDEND)]

i = [0,0,0,0,0]
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    time_slot = (int)((row[4] - min_time) / time_range_length)
    
    data_list[time_slot].append(row[1:4])
    
    row_index[time_slot].append(i[time_slot])
    col_index[time_slot].append(row[1] -1)
    rate_list[time_slot].append(1)
    row_index[time_slot].append(i[time_slot])
    col_index[time_slot].append(row[2]-1 + n_users)
    rate_list[time_slot].append(1)
    vector_b[time_slot].append(row[3])
    
    i[time_slot] += 1

matrix_A_list = []
for j in range(0, TIME_RANGE_DIVIDEND):
    row = np.array(row_index[j])
    col = np.array(col_index[j])
    rate = np.array(rate_list[j])
    matrix_A_list.append(sps.coo_matrix((rate, (row, col)), shape=(len(row)/2,n_users+n_items)))

# clear temporary data
del row_index[:]
del row_index
del col_index[:]
del col_index
del rate_list[:]
del rate_list


# print(matrix_A)

In [67]:
all_mean_list = []

for i in range(0, TIME_RANGE_DIVIDEND): 
    temp_list = []
    for row in data_list[i]:
        temp_list.append(row[2])
    temp_np_arr = np.array(temp_list)
    all_mean_list.append(np.mean(temp_np_arr[temp_np_arr!=0]))
    vector_b[i][:] = [x - all_mean_list[i] for x in vector_b[i]]

print("all_mean_list: ", all_mean_list)
# print(vector_b)
# print(vector_b)
# print(vector_b[:5])

# print(vector_b[:10])

[3.5714902807775379, 3.5621575342465754, 3.4556953729191244, 3.4428491216812387, 3.5952079866888518]


In [68]:
bu_and_bi_list = []

## least square with regularization
for i in range(0, TIME_RANGE_DIVIDEND):
    print(matrix_A_list[i].toarray().shape)
    print(len(vector_b[i]))
    return_values = lsqr(matrix_A_list[i].toarray(), vector_b[i], damp = 1)
    bu_and_bi = return_values[0]
    print(bu_and_bi[:5])
    bu_and_bi_list.append(bu_and_bi)

(13890, 2625)
13890
[-0.12970215  0.          0.          0.         -0.65115339]
(23360, 2625)
23360
[-0.29571164  0.          0.          0.         -0.90746737]
(15258, 2625)
15258
[-0.76711151  0.          0.          0.          0.        ]
(12467, 2625)
12467
[ 0.10602819 -0.04959629 -0.49728202  0.          0.        ]
(15025, 2625)
15025
[ 0.00617896  0.          0.          0.67091029  0.        ]


In [69]:
def rmse(pred, actual):
    '''计算预测结果的rmse'''
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return np.sqrt(mean_squared_error(pred, actual))

In [70]:
def predict_naive(user, item, timestamp):
    time_slot = (int)((timestamp - min_time) / time_range_length)
    prediction = all_mean_list[time_slot] + bu_and_bi_list[time_slot][user] + bu_and_bi_list[time_slot][n_users -1 + item]
    return prediction

In [93]:
print('载入测试集...')
test_df = pd.read_csv(testset_file, sep='\t', names=names)
test_df.head()
predictions = []
targets = []
print('测试集大小为 %d' % len(test_df))
print('采用Naive算法进行预测...')
for row in test_df.itertuples():
    user, item, actual, timestamp = row[1]-1, row[2]-1, row[3], row[4]
    predictions.append(predict_naive(user, item, timestamp))
    targets.append(actual)

print('测试结果的rmse为 %.4f' % rmse(np.array(predictions), np.array(targets)))

载入测试集...
测试集大小为 20000
采用Naive算法进行预测...
测试结果的rmse为 1.0957


In [105]:
# rating to rating_delta, need to do the shift, prepare for the top-k neighbor
for row in df.itertuples():
    time_slot = (int)((row[4] - min_time) / time_range_length)
    ratings[row[1]-1, row[2]-1] = row[3] - \
        (all_mean_list[time_slot] + bu_and_bi_list[time_slot][row[1]-1] + bu_and_bi_list[time_slot][n_users -1 + row[2]-1])

In [109]:
def cos_distance(ratings, kind, epsilon=1e-9):
    '''利用Cosine距离计算相似度'''
    '''epsilon: 防止Divide-by-zero错误，进行矫正'''
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

item_similarity = cos_distance(ratings, 'item')
print(np.round_(item_similarity[:10,:10], 3))

[[ 1.    -0.121 -0.061  0.079  0.011  0.055  0.075  0.072 -0.002 -0.044]
 [-0.121  1.     0.006 -0.117 -0.024 -0.021 -0.099  0.03  -0.056 -0.002]
 [-0.061  0.006  1.    -0.066  0.009  0.01  -0.065 -0.09  -0.061 -0.01 ]
 [ 0.079 -0.117 -0.066  1.    -0.074  0.006  0.076  0.077  0.025  0.008]
 [ 0.011 -0.024  0.009 -0.074  1.    -0.028 -0.039 -0.036  0.013 -0.038]
 [ 0.055 -0.021  0.01   0.006 -0.028  1.     0.016 -0.05   0.05  -0.005]
 [ 0.075 -0.099 -0.065  0.076 -0.039  0.016  1.     0.018  0.084 -0.008]
 [ 0.072  0.03  -0.09   0.077 -0.036 -0.05   0.018  1.    -0.027  0.029]
 [-0.002 -0.056 -0.061  0.025  0.013  0.05   0.084 -0.027  1.    -0.014]
 [-0.044 -0.002 -0.01   0.008 -0.038 -0.005 -0.008  0.029 -0.014  1.   ]]


In [122]:
import math

'''incorporate top-k neighbor, and improved baseline predictor'''
def predict_topkCF(user, item, timestamp, k=30):
    '''top-k CF算法,以item-item协同过滤为基础，结合baseline,预测rating'''
    nzero = ratings[user].nonzero()[0]
    baseline = []
    time_slot = (int)((timestamp - min_time) / time_range_length)
    
    for item_num in nzero:    
        temp = bu_and_bi_list[time_slot][user] + bu_and_bi_list[time_slot][n_users -1 + item_num]
        baseline.append(temp)
    
    choices = nzero[item_similarity[item, nzero].argsort()[::-1][:k]]
    
    distance_sum = 0
    for choice in choices:
        distance_sum += math.fabs(item_similarity[item, choice])
        
    neighbor_contribution = 0
    for choice in choices:
        neighbor_contribution += (item_similarity[item, choice]/distance_sum)*ratings[user, choice]
    
    item_baseline = all_mean_list[time_slot] + bu_and_bi_list[time_slot][user] + \
        bu_and_bi_list[time_slot][n_users -1 + item]
        
    prediction = item_baseline + neighbor_contribution
    
    if prediction > 5: prediction = 5
    if prediction < 0.5: prediction = 0.5
    return prediction 

In [123]:
print('载入测试集...')
test_df = pd.read_csv(testset_file, sep='\t', names=names)
test_df.head()
predictions = []
targets = []
print('测试集大小为 %d' % len(test_df))
print('采用结合baseline的item-item协同过滤算法进行预测...')
for row in test_df.itertuples():
    user, item, actual, timestamp = row[1]-1, row[2]-1, row[3], row[4]
    predictions.append(predict_topkCF(user, item, timestamp))
    targets.append(actual)

print('测试结果的rmse为 %.4f' % rmse(np.array(predictions), np.array(targets)))
print()

载入测试集...
测试集大小为 20000
采用结合baseline的item-item协同过滤算法进行预测...
测试结果的rmse为 0.9765

